# Learning: algorithms, objectives, and assumptions
(c) Deniz Yuret 2019

In this notebook we will analyze three classic learning algorithms.
* **Perceptron:** ([Rosenblatt, 1957](https://en.wikipedia.org/wiki/Perceptron)) a neuron model trained with a simple algorithm that updates model weights using the input when the prediction is wrong.
* **Adaline:** ([Widrow and Hoff, 1960](https://en.wikipedia.org/wiki/ADALINE)) a neuron model trained with a simple algorithm that updates model weights using the error multiplied by the input (aka least mean square (LMS), delta learning rule, or the Widrow-Hoff rule).
* **Softmax classification:** ([Cox, 1958](https://en.wikipedia.org/wiki/Multinomial_logistic_regression)) a multiclass generalization of the logistic regression model from statistics (aka multinomial LR, softmax regression, maxent classifier etc.).

We look at these learners from different perspectives:
* **Algorithm:** First we ask only **how** the learner works, i.e. how it changes after observing each example.
* **Objectives:** Next we ask **what** objective guides the algorithm, whether it is optimizing a particular objective function, and whether we can use a generic *optimization algorithm* instead.
* **Assumptions:** Finally we ask **why** we think this algorithm makes sense, what prior assumptions does this imply and whether we can use *probabilistic inference* for optimal learning. (TODO)

In [1]:
using Knet, MLDatasets, Statistics, LinearAlgebra, Random
ARRAY = Array{Float32}
ENV["COLUMNS"] = 72

72

### Data

In [2]:
xtrn, ytrn = MNIST.traindata(eltype(ARRAY)) 
xtst, ytst = MNIST.testdata(eltype(ARRAY))
xtrn, xtst = ARRAY(mat(xtrn)), ARRAY(mat(xtst))
onehot(y) = (m=zeros(eltype(ARRAY),10,length(y)); for i in 1:length(y); m[y[i]==0 ? 10 : y[i],i]=1; end; ARRAY(m))
ytrn, ytst = onehot(ytrn), onehot(ytst);

In [3]:
println.(summary.((xtrn, ytrn, xtst, ytst)));

784×60000 Array{Float32,2}
10×60000 Array{Float32,2}
784×10000 Array{Float32,2}
10×10000 Array{Float32,2}


In [4]:
NTRN,NTST,XDIM,YDIM = size(xtrn,2), size(xtst,2), size(xtrn,1), size(ytrn,1)

(60000, 10000, 784, 10)

### Prediction

In [5]:
# Model weights
w = ARRAY(randn(YDIM,XDIM))

10×784 Array{Float32,2}:
 -0.01215     0.628198  -0.740396  …  -0.630068   0.280267  -0.940353
 -0.863612   -1.92136    1.39318       0.879314   0.247981   0.348589
 -0.0308054   0.268047   0.605304      0.610452  -0.276971   0.757551
 -2.79762    -0.582162   0.268582     -0.607423   1.61061    1.35039
 -0.153525   -0.196723   0.282439      0.898576  -0.985867   0.146549
  0.841244    0.166082   0.484519  …  -0.893788   0.588566  -0.330542
 -0.0370356  -0.675147  -0.373608      1.81982    0.687714  -0.405147
 -2.33272    -0.9257    -0.384133      0.338714  -0.491783   0.110337
 -1.09294    -0.211454  -0.236161     -1.79776    1.93232    0.155101
  0.0560821   0.667874   1.21667       0.959533   2.63739   -0.616039

In [6]:
# Class scores
w * xtrn

10×60000 Array{Float32,2}:
 12.0088     12.6426     6.83006   …  10.5872    16.2395    4.72857
 -0.977543    8.03122   -0.653524      0.724494  12.2396    1.17001
 -6.0617     -3.13611    5.28498      -0.447501  -8.94851   5.51493
 14.084      12.459      7.1651       15.484     10.3361    6.92621
  3.26313    -2.96441  -16.6912        6.37751   -7.10098   2.64587
  6.07298   -15.7869    -0.260126  …  -2.72658   -3.23432   0.259716
  1.69763     8.43725    6.54435       4.119      6.72252  -3.45105
  2.6587     -3.62504    6.70296       0.625211  -1.21695   9.72003
 -5.79982    -7.12983  -13.4711       -7.95972    6.02882   4.25041
 -2.86784    -5.86244   -3.21701       0.428129  -1.10273  -0.971755

In [7]:
# Predictions
[ argmax(w * xtrn[:,i]) for i in 1:NTRN ]'

1×60000 Adjoint{Int64,Array{Int64,1}}:
 4  1  4  8  9  8  1  4  9  1  8  …  4  4  1  1  8  3  1  8  4  1  8

In [8]:
# Correct answers
[ argmax(ytrn[:,i]) for i in 1:NTRN ]'

1×60000 Adjoint{Int64,Array{Int64,1}}:
 5  10  4  1  9  2  1  3  1  4  3  …  8  9  2  9  5  1  8  3  5  6  8

In [9]:
# Accuracy
acc(w,x,y) = mean(argmax(w * x, dims=1) .== argmax(y, dims=1))
acc(w,xtrn,ytrn), acc(w,xtst,ytst)

(0.07835, 0.0769)

## Algorithms

In [10]:
# Training loop
function train(algo,x,y,T=2^20)
    w = ARRAY(zeros(size(y,1),size(x,1)))
    nexamples = size(x,2)
    nextprint = 1
    for t = 1:T
        i = rand(1:nexamples)
        algo(w, x[:,i], y[:,i])  # <== this is where w is updated
        if t == nextprint
            println((iter=t, accuracy=acc(w,x,y), wnorm=norm(w)))
            nextprint = min(2t,T)
        end
    end
    w
end

train (generic function with 2 methods)

### Perceptron

In [11]:
function perceptron(w,x,y)
    guess = argmax(w * x)
    class = argmax(y)
    if guess != class
        w[class,:] .+= x
        w[guess,:] .-= x
    end
end

perceptron (generic function with 1 method)

In [12]:
# (iter = 1048576, accuracy = 0.8950333333333333, wnorm = 1321.2463f0) in 7 secs
@time wperceptron = train(perceptron,xtrn,ytrn);

(iter = 1, accuracy = 0.0993, wnorm = 16.183395f0)
(iter = 2, accuracy = 0.14571666666666666, wnorm = 18.594425f0)
(iter = 4, accuracy = 0.12638333333333332, wnorm = 21.674007f0)
(iter = 8, accuracy = 0.24383333333333335, wnorm = 30.619133f0)
(iter = 16, accuracy = 0.32233333333333336, wnorm = 38.3449f0)
(iter = 32, accuracy = 0.40315, wnorm = 48.85941f0)
(iter = 64, accuracy = 0.48985, wnorm = 61.05146f0)
(iter = 128, accuracy = 0.4500166666666667, wnorm = 87.74915f0)
(iter = 256, accuracy = 0.6148, wnorm = 111.76928f0)
(iter = 512, accuracy = 0.69185, wnorm = 140.70638f0)
(iter = 1024, accuracy = 0.7066333333333333, wnorm = 184.62056f0)
(iter = 2048, accuracy = 0.7945333333333333, wnorm = 221.71835f0)
(iter = 4096, accuracy = 0.8195, wnorm = 273.88263f0)
(iter = 8192, accuracy = 0.8366166666666667, wnorm = 333.44574f0)
(iter = 16384, accuracy = 0.8612833333333333, wnorm = 397.18716f0)
(iter = 32768, accuracy = 0.80065, wnorm = 484.5039f0)
(iter = 65536, accuracy = 0.8796833333333334,

### Adaline

In [13]:
function adaline(w,x,y; lr=0.0001)
    error = w * x - y
    w .-= lr * error * x'
end

adaline (generic function with 1 method)

In [14]:
# (iter = 1048576, accuracy = 0.8523, wnorm = 1.2907721f0) in 31 secs with lr=0.0001
@time wadaline = train(adaline,xtrn,ytrn);

(iter = 1, accuracy = 0.10441666666666667, wnorm = 0.0010784594f0)
(iter = 2, accuracy = 0.10441666666666667, wnorm = 0.0018062098f0)
(iter = 4, accuracy = 0.14796666666666666, wnorm = 0.0023069703f0)
(iter = 8, accuracy = 0.1129, wnorm = 0.0031464882f0)
(iter = 16, accuracy = 0.1797, wnorm = 0.0055435537f0)
(iter = 32, accuracy = 0.25838333333333335, wnorm = 0.008615678f0)
(iter = 64, accuracy = 0.1327, wnorm = 0.0146190785f0)
(iter = 128, accuracy = 0.28781666666666667, wnorm = 0.02509437f0)
(iter = 256, accuracy = 0.6364, wnorm = 0.040473588f0)
(iter = 512, accuracy = 0.67785, wnorm = 0.065088175f0)
(iter = 1024, accuracy = 0.7113166666666667, wnorm = 0.10240222f0)
(iter = 2048, accuracy = 0.7436166666666667, wnorm = 0.1651955f0)
(iter = 4096, accuracy = 0.7975333333333333, wnorm = 0.25398782f0)
(iter = 8192, accuracy = 0.7949, wnorm = 0.35792586f0)
(iter = 16384, accuracy = 0.8295, wnorm = 0.46056572f0)
(iter = 32768, accuracy = 0.8382666666666667, wnorm = 0.56489986f0)
(iter = 655

### Softmax classifier

In [15]:
function softmax(w,x,y; lr=0.01)
    probs = exp.(w * x)
    probs ./= sum(probs)
    error = probs - y
    w .-= lr * error * x'
end

softmax (generic function with 1 method)

In [16]:
# (iter = 1048576, accuracy = 0.9242166666666667, wnorm = 26.523603f0) in 32 secs with lr=0.01
@time wsoftmax = train(softmax,xtrn,ytrn);

(iter = 1, accuracy = 0.09871666666666666, wnorm = 0.09837641f0)
(iter = 2, accuracy = 0.09871666666666666, wnorm = 0.18767525f0)
(iter = 4, accuracy = 0.10088333333333334, wnorm = 0.24415228f0)
(iter = 8, accuracy = 0.19958333333333333, wnorm = 0.27368906f0)
(iter = 16, accuracy = 0.15525, wnorm = 0.44634444f0)
(iter = 32, accuracy = 0.4477833333333333, wnorm = 0.5210072f0)
(iter = 64, accuracy = 0.5075, wnorm = 0.81101984f0)
(iter = 128, accuracy = 0.5968333333333333, wnorm = 1.2767634f0)
(iter = 256, accuracy = 0.7307, wnorm = 1.9046426f0)
(iter = 512, accuracy = 0.72075, wnorm = 2.7310653f0)
(iter = 1024, accuracy = 0.8428666666666667, wnorm = 3.651138f0)
(iter = 2048, accuracy = 0.8595666666666667, wnorm = 4.6969028f0)
(iter = 4096, accuracy = 0.86895, wnorm = 5.87642f0)
(iter = 8192, accuracy = 0.89265, wnorm = 7.225823f0)
(iter = 16384, accuracy = 0.9005666666666666, wnorm = 8.691754f0)
(iter = 32768, accuracy = 0.9052166666666667, wnorm = 10.411587f0)
(iter = 65536, accuracy = 

## Objectives

In [17]:
# Training via optimization
function optimize(loss,x,y; lr=0.1, iters=2^20)
    w = Param(ARRAY(zeros(size(y,1),size(x,1))))
    nexamples = size(x,2)
    nextprint = 1
    for t = 1:iters
        i = rand(1:nexamples)
        L = @diff loss(w, x[:,i], y[:,i])
        ∇w = grad(L,w)
        w .-= lr * ∇w
        if t == nextprint
            println((iter=t, accuracy=acc(w,x,y), wnorm=norm(w)))
            nextprint = min(2t,iters)
        end
    end
    w
end

optimize (generic function with 1 method)

### Perceptron minimizes the score difference between the correct class and the prediction

In [18]:
function perceptronloss(w,x,y)
    score = w * x
    guess = argmax(score)
    class = argmax(y)
    score[guess] - score[class]
end

perceptronloss (generic function with 1 method)

In [19]:
AutoGrad.@zerograd argmax(x;dims=:) # to be fixed in AutoGrad

In [20]:
# (iter = 1048576, accuracy = 0.8908833333333334, wnorm = 1322.4888f0) in 62 secs with lr=1
@time wperceptron2 = optimize(perceptronloss,xtrn,ytrn,lr=1);

(iter = 1, accuracy = 0.09751666666666667, wnorm = 17.802845f0)
(iter = 2, accuracy = 0.11656666666666667, wnorm = 19.03113f0)
(iter = 4, accuracy = 0.0977, wnorm = 23.478838f0)
(iter = 8, accuracy = 0.1641, wnorm = 25.38451f0)
(iter = 16, accuracy = 0.21983333333333333, wnorm = 35.316483f0)
(iter = 32, accuracy = 0.2434, wnorm = 46.611935f0)
(iter = 64, accuracy = 0.43803333333333333, wnorm = 67.370865f0)
(iter = 128, accuracy = 0.57065, wnorm = 87.302666f0)
(iter = 256, accuracy = 0.63065, wnorm = 114.32892f0)
(iter = 512, accuracy = 0.73915, wnorm = 145.60088f0)
(iter = 1024, accuracy = 0.73595, wnorm = 173.54251f0)
(iter = 2048, accuracy = 0.7659166666666667, wnorm = 218.51064f0)
(iter = 4096, accuracy = 0.80205, wnorm = 279.03473f0)
(iter = 8192, accuracy = 0.8284833333333333, wnorm = 334.0729f0)
(iter = 16384, accuracy = 0.8654666666666667, wnorm = 413.40176f0)
(iter = 32768, accuracy = 0.8414166666666667, wnorm = 495.76038f0)
(iter = 65536, accuracy = 0.8770666666666667, wnorm =

### Adaline minimizes the squared error

In [21]:
function quadraticloss(w,x,y)
    0.5 * sum(abs2, w * x - y)
end

quadraticloss (generic function with 1 method)

In [22]:
# (iter = 1048576, accuracy = 0.8498333333333333, wnorm = 1.2882874f0) in 79 secs with lr=0.0001
@time wadaline2 = optimize(quadraticloss,xtrn,ytrn,lr=0.0001);

(iter = 1, accuracy = 0.09736666666666667, wnorm = 0.0009124075f0)
(iter = 2, accuracy = 0.11793333333333333, wnorm = 0.0011352332f0)
(iter = 4, accuracy = 0.14368333333333333, wnorm = 0.0019526349f0)
(iter = 8, accuracy = 0.14278333333333335, wnorm = 0.003078862f0)
(iter = 16, accuracy = 0.158, wnorm = 0.0047181337f0)
(iter = 32, accuracy = 0.12331666666666667, wnorm = 0.008533138f0)
(iter = 64, accuracy = 0.3027666666666667, wnorm = 0.015150967f0)
(iter = 128, accuracy = 0.22838333333333333, wnorm = 0.026578516f0)
(iter = 256, accuracy = 0.48543333333333333, wnorm = 0.042260554f0)
(iter = 512, accuracy = 0.6769166666666667, wnorm = 0.06585408f0)
(iter = 1024, accuracy = 0.7186, wnorm = 0.10273933f0)
(iter = 2048, accuracy = 0.7477166666666667, wnorm = 0.16474141f0)
(iter = 4096, accuracy = 0.7714166666666666, wnorm = 0.2536855f0)
(iter = 8192, accuracy = 0.8083833333333333, wnorm = 0.3555469f0)
(iter = 16384, accuracy = 0.8217333333333333, wnorm = 0.46040946f0)
(iter = 32768, accurac

### Softmax classifier maximizes the probabilities of correct answers
(or minimizes negative log likelihood, aka cross-entropy or softmax loss)

In [23]:
function negloglik(w,x,y)
    probs = exp.(w * x)
    probs = probs / sum(probs)
    class = argmax(y)
    -log(probs[class])
end

negloglik (generic function with 1 method)

In [24]:
# (iter = 1048576, accuracy = 0.9283833333333333, wnorm = 26.593485f0) in 120 secs with lr=0.01
@time wsoftmax2 = optimize(negloglik,xtrn,ytrn,lr=0.01);

(iter = 1, accuracy = 0.11236666666666667, wnorm = 0.05294757f0)
(iter = 2, accuracy = 0.0994, wnorm = 0.11926413f0)
(iter = 4, accuracy = 0.18665, wnorm = 0.16653569f0)
(iter = 8, accuracy = 0.2708, wnorm = 0.2787828f0)
(iter = 16, accuracy = 0.25911666666666666, wnorm = 0.37261048f0)
(iter = 32, accuracy = 0.35436666666666666, wnorm = 0.51743394f0)
(iter = 64, accuracy = 0.49038333333333334, wnorm = 0.78246355f0)
(iter = 128, accuracy = 0.61035, wnorm = 1.2495269f0)
(iter = 256, accuracy = 0.7281833333333333, wnorm = 1.8780376f0)
(iter = 512, accuracy = 0.7697166666666667, wnorm = 2.7066066f0)
(iter = 1024, accuracy = 0.8248, wnorm = 3.670761f0)
(iter = 2048, accuracy = 0.8471333333333333, wnorm = 4.6900873f0)
(iter = 4096, accuracy = 0.8789166666666667, wnorm = 5.87028f0)
(iter = 8192, accuracy = 0.88915, wnorm = 7.179222f0)
(iter = 16384, accuracy = 0.8981666666666667, wnorm = 8.627161f0)
(iter = 32768, accuracy = 0.9089666666666667, wnorm = 10.263525f0)
(iter = 65536, accuracy = 0